In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hadoop fs -rm -f -r tmp;

Deleted tmp


In [3]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 9.32 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.954 seconds


In [4]:
%%hive
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.173 seconds


In [5]:
%%hive
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.059 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.066 seconds


In [6]:
%%hive
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 0.766 seconds


In [10]:
%%hive
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
    SELECT
        concat(tbl0.c1,',',
        tbl0.c2,',',
        tbl1.c4[tbl0.c2])
    FROM
        tbl0
    JOIN (
        SELECT
            c1,
            c4
        FROM
            tbl1
        ) tbl1
    ON
        (tbl0.c1 = tbl1.c1);

DROP TABLE IF EXISTS salida;
OK
Time taken: 0.302 seconds
CREATE TABLE salida
AS
    SELECT
        concat(tbl0.c1,',',
        tbl0.c2,',',
        tbl1.c4[tbl0.c2])
    FROM
        tbl0
    JOIN (
        SELECT
            c1,
            c4
        FROM
            tbl1
        ) tbl1
    ON
        (tbl0.c1 = tbl1.c1);
Query ID = root_20200120001402_2e972e2f-104a-4848-a655-c16d70215453
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2020-01-20 00:14:09	Starting to launch local task to process map join;	maximum memory = 477626368
2020-01-20 00:

In [11]:
%%hive
--SELECT * 
--FROM salida;

--SELECT * 
--FROM salida;


In [12]:
%%hive
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;

INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT
    *
FROM
    salida;
Query ID = root_20200120001507_0986ba58-e0c3-491d-aa64-ea23b3cca83d
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1579469532371_0010, Tracking URL = http://92c98434d4d4:8088/proxy/application_1579469532371_0010/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579469532371_0010
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-01-20 00:15:14,943 Stage-1 map = 0%,  reduce = 0%
2020-01-20 00:15:19,318 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.16 sec
MapReduce Total cumulative CPU time: 1 seconds 160 msec
Ended Job = job_1579469532371_0010
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/user/root/tmp/000000_

In [13]:
!hadoop fs -copyToLocal ./tmp/000000_0 output

In [14]:
%hive_quit